In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Exploration

In [ ]:
directory = os.getcwd()
df = pd.read_excel(directory + "\\online_retail_II.xlsx", sheet_name = 0)
df.head()

In [ ]:
df.info()

## Considerations:
1. Customer ID has a significant number of missing values
2. InvoiceDate already has the correct data type

In [ ]:
df.describe()

## Considerations:
1. min(Quantity) and min(Price) are negative

In [ ]:
df.describe(include = 'O')

In [ ]:
# Investigating missing values of Customer IDs
df[df['Customer ID'].isna()].head()

## Considerations:
1. It's really hard to get the actual Customer ID if it's missing (even if the transaction seems legit), so this data will be excluded from the analysis

In [ ]:
# Investigating negative quantities
df[df['Quantity'] < 0].head()

From the variables description in the dataset, it's clear that the letter 'C' at the beginning of the invoice indicates a cancellation

In [ ]:
# Investigating invoice types
df['Invoice'] = df['Invoice'].astype('str')
df[df['Invoice'].str.match('^\\d{6}$') == False] # Regex: matching exactly 6 digits

In [ ]:
# Are there any other letter than the 'C'?
df['Invoice'].replace('[0-9]', '', regex = True).unique() # Regex: replacing all digits with an empty string

In [ ]:
# Investigating the 'A'
df[df['Invoice'].str.startswith('A')]

These transactions seem to be accounting operations and they are very few, so they'll be removed

Moving to the 'StockCode', from the variables description in the dataset it should a 5 digits code. Let's be sure about that

In [ ]:
df['StockCode'] = df['StockCode'].astype('str')
df[df['StockCode'].str.match('^\\d{5}$') == False]

There appear to be letters after 5 digits. These transactions appear to be legit, but I am investigating into these letters since they're not mentioned in the dataset description

In [ ]:
# Regex: 5 digits followed by letters eventually repeated
unique_codes = df[(df['StockCode'].str.match('^\\d{5}$') == False) & (df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == False)]['StockCode'].unique()
for code in unique_codes:
    filtered_df = df[df['StockCode'].str.contains(f'^{code}')]
    print(f"StockCode '{code}':")
    print(filtered_df.head())
    print("\n")

## Summary:
| Stock Code          | Description                                                            | Action                  |
|---------------------|------------------------------------------------------------------------|-------------------------|
| DCGS               | Looks valid, some quantities are negative though and customer ID is null | Exclude |
| D                  | Looks valid, represents discount values                                | Exclude  |
| DOT                | Looks valid, represents postage charges                                | Exclude  |
| M or m             | Looks valid, represents manual transactions                            | Exclude  |
| C2                 | Carriage transaction - not sure what this means                        | Exclude  |
| C3                 | Not sure, only 1 transaction                                           | Exclude                 |
| BANK CHARGES or B  | Bank charges                                                           | Exclude  |
| S                  | Samples sent to customer                                               | Exclude  |
| TESTXXX            | Testing data, not valid                                                | Exclude  |
| gift__XXX          | Purchases with gift cards, might be interesting for another analysis, but no customer data | Exclude |
| PADS               | Looks like a legit stock code for padding                              | Include                 |
| SP1002             | Looks like a special request item, only 2 transactions, 3 look legit, 1 has 0 pricing | Exclude |
| AMAZONFEE          | Looks like fees for Amazon shipping or something                       | Exclude  |
| ADJUSTX            | Looks like manual account adjustments by admins                        | Exclude  |

# Data Cleaning

In [ ]:
clean_df = df.copy()

In [ ]:
clean_df['Invoice'] = clean_df['Invoice'].astype('str')
mask = (
    clean_df['Invoice'].str.match('^\\d{6}$') == True
)
clean_df = clean_df[mask]
clean_df

In [ ]:
clean_df["StockCode"] = clean_df["StockCode"].astype("str")
'''
I want StockCodes with 5 digits, or 5 digits followed by letters, or 'PADS' StockCodes as declared previously
'''
mask = (
    (clean_df["StockCode"].str.match("^\\d{5}$") == True)
    | (clean_df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == True)
    | (clean_df["StockCode"].str.match("^PADS$") == True)
)
clean_df = clean_df[mask]
clean_df

In [ ]:
clean_df.dropna(subset = ['Customer ID'], inplace=True)

In [ ]:
clean_df.describe()

## Considerations:
The negative prices have been fixed as well as negative quantity. There are prices = 0

In [ ]:
clean_df[clean_df['Price'] == 0]

0 prices items should not be included in my analysis since they're probably gifts or something like that

In [ ]:
clean_df = clean_df[clean_df['Price'] > 0]
clean_df.describe()

There is a really small price, but I think it is okay

## Data Cleaning summary
How much data we still have after the Data Cleaning step?

In [ ]:
print(str((len(clean_df) / len(df)) * 100) + '%')

# Feature Engineering

In [ ]:
clean_df = clean_df.copy()
clean_df[ 'SalesTotal'] = clean_df['Quantity'] * clean_df['Price']
clean_df.head()

In [ ]:
agg_df = clean_df.groupby('Customer ID', as_index = False).agg(
    Total = ('SalesTotal', 'sum'),
    Frequency = ('Invoice', 'nunique'),
    LastInvoiceDate = ('InvoiceDate', 'max')
)
agg_df.head()

In [ ]:
max_invoice_date = agg_df['LastInvoiceDate'].max()
agg_df['Recency'] = (max_invoice_date - agg_df['LastInvoiceDate']).dt.days
agg_df.head()

In [ ]:
# Are there any outliers? They may affect the clustering
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(agg_df['Total'], bins=100, color='skyblue', edgecolor='black')
plt.title('Total Distribution')
plt.xlabel('Total')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
plt.hist(agg_df['Frequency'], bins=100, color='lightgreen', edgecolor='black')
plt.title('Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
plt.hist(agg_df['Recency'], bins=20, color='salmon', edgecolor='black')
plt.title('Recency Distribution')
plt.xlabel('Recency')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data = agg_df['Total'], color='skyblue')
plt.title('Total Boxplot')
plt.xlabel('Total')

plt.subplot(1, 3, 2)
sns.boxplot(data = agg_df['Frequency'], color='lightgreen')
plt.title('Frequency Boxplot')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.boxplot(data = agg_df['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

Few outliers in 'Recency', but many in 'Total' and 'Frequency'.

Since the outliers in the first two charts represent the most valuable clients, they should not be removed. I'll perform a separate analysis on them.

In [ ]:
T_Q1 = agg_df['Total'].quantile(0.25)
T_Q3 = agg_df['Total'].quantile(0.75)
T_IQR = T_Q3 - T_Q1

# Quantile outlier definition
total_outliers = agg_df[(agg_df['Total'] > (T_Q3 + 1.5 * T_IQR)) | (agg_df['Total'] < (T_Q1 - 1.5 * T_IQR))].copy()
total_outliers.describe()

In [ ]:
F_Q1 = agg_df['Frequency'].quantile(0.25)
F_Q3 = agg_df['Frequency'].quantile(0.75)
F_IQR = F_Q3 - F_Q1
frequency_outliers = agg_df[(agg_df['Frequency'] > (F_Q3 + 1.5 * F_IQR)) | (agg_df['Frequency'] < (F_Q1 - 1.5 * F_IQR))].copy()
frequency_outliers.describe()

In [ ]:
# The Total and Frequency outliers may have values in common, so let's filter them out
non_outliers = agg_df[(~agg_df.index.isin(total_outliers.index)) & (~agg_df.index.isin(frequency_outliers.index))]
non_outliers.describe()

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data = non_outliers['Total'], color='skyblue')
plt.title('Total Boxplot')
plt.xlabel('Total')

plt.subplot(1, 3, 2)
sns.boxplot(data = non_outliers['Frequency'], color='lightgreen')
plt.title('Frequency Boxplot')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.boxplot(data = non_outliers['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = fig.add_subplot(projection = '3d')
scatter = ax.scatter(non_outliers['Total'], non_outliers['Frequency'], non_outliers['Recency'])

ax.set_xlabel('Total')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D scatter plot of customer data')
plt.tight_layout()
plt.show()

It is clear the difference of scales. Indeed, 'Total' may have a bigger weight during clustering.

Standard scaling transforms the features of your data to have a mean of 0 and a standard deviation of 1, ensuring that each feature contributes equally to the analysis (assuming a normal distribution).


$$z = \frac{x - \mu}{\sigma}$$


Where:

- $ z $ is the standardized value,
- $ x $ is the original value,
- $ \mu $ is the mean of the feature,
- $ \sigma $ is the standard deviation of the feature.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(non_outliers[['Total', 'Frequency', 'Recency']])
scaled_data_df = pd.DataFrame(scaled_data, index = non_outliers.index, columns=("Total", "Frequency", "Recency"))
scaled_data_df

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = fig.add_subplot(projection = '3d')
scatter = ax.scatter(scaled_data_df['Total'], scaled_data_df['Frequency'], scaled_data_df['Recency'])

ax.set_xlabel('Total')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D scatter plot of customer data (scaled)')
plt.tight_layout()
plt.show()

# KMeans Clustering

### Silhouette Score

$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$

Where:
- $s(i)$ is the silhouette score for a single sample $i$,
- $a(i)$ is the average distance between $i$ and all other points in the same cluster,
- $b(i)$ is the minimum average distance between $i$ and all points in the nearest cluster to which $i$ does not belong.

The silhouette score ranges between $[-1, 1]$, where a higher value indicates more distinct clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Set the number of cores explicitly to avoid a warning
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

max_k = 20
inertia = []
silhouette_scores = []
k_values = range(2, max_k + 1)

for k in k_values:
    kmeans = KMeans(n_clusters = k, random_state = 0, max_iter = 1000)
    cluster_labels = kmeans.fit_predict(scaled_data_df)
    
    # Calculate silhouette score
    sil_score = silhouette_score(scaled_data_df, cluster_labels)
    silhouette_scores.append(sil_score)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(15, 5))

# Inertia plot
plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker = 'o')
plt.title('KMeans Inertia for Different Values of k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_values)
plt.grid(True)

# Silhouette score plot
plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, marker = 'o', color = 'orange')
plt.title('Silhouette Scores for Different Values of k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(k_values)
plt.grid(True)

plt.tight_layout()
plt.show()

To apply the Helbow Method, we need to locate the Helbow of the Inertia function. The Silhouette Score tells us that the higher value is scored at k = 3. Therefore, at this value the clusters will be less overlapped. Also, it shown that further k values don't make the Inertia Score any higher

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state = 0, max_iter = 1000)
cluster_labels = kmeans.fit_predict(scaled_data_df)
cluster_labels

In [ ]:
non_outliers = non_outliers.copy()
non_outliers['Cluster'] = cluster_labels
non_outliers

In [ ]:
cluster_colors = {0: '#1f77b4',  # Blue
                  1: '#ff7f0e',  # Orange
                  2: '#2ca02c',  # Green
                 }
colors = non_outliers['Cluster'].map(cluster_colors)

fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(projection = '3d')

scatter = ax.scatter(non_outliers['Total'], 
                     non_outliers['Frequency'], 
                     non_outliers['Recency'], 
                     c = colors,
                     marker = 'o')

ax.set_xlabel('Total')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data by Cluster')

plt.show()

In [ ]:
plt.figure(figsize = (12, 18))

plt.subplot(3, 1, 1)
sns.violinplot(x = non_outliers['Cluster'], y = non_outliers['Total'], palette = cluster_colors, hue = non_outliers["Cluster"])
sns.violinplot(y = non_outliers['Total'], color = 'gray', linewidth = 1.0) # Violin plot of all unclustered data
plt.title('Total by Cluster')
plt.ylabel('Total')

plt.subplot(3, 1, 2)
sns.violinplot(x = non_outliers['Cluster'], y = non_outliers['Frequency'], palette = cluster_colors, hue = non_outliers["Cluster"])
sns.violinplot(y = non_outliers['Frequency'], color =  'gray', linewidth = 1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')


plt.subplot(3, 1, 3)
sns.violinplot(x = non_outliers['Cluster'], y = non_outliers['Recency'], palette = cluster_colors, hue = non_outliers["Cluster"])
sns.violinplot(y = non_outliers['Recency'], color = 'gray', linewidth = 1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')

plt.tight_layout()
plt.show()

## Cluster Identification and Actions

### **Cluster 0 (Blue): "New Shoppers"**
- **Rationale**:  
  This cluster represents customers who purchase not very frequently, but very recently. They may be new customers who are exploring the products or services.  
- **Action**:  
  - Provide excellent customer service to make a positive first impression.  
  - Offer incentives such as discounts or promotions to encourage frequent purchases and build loyalty.

---

### **Cluster 1 (Orange): "Re-engage"**
- **Rationale**:  
  This cluster represents customers who were highly active in the past (frequent and high-value purchases) but have become inactive recently. They may have lost interest or been drawn away by competitors.  
- **Action**:  
  - Launch targeted marketing campaigns to regain their interest.  
  - Offer special discounts or personalized promotions to entice them back.  
  - Send reminders, newsletters, or product updates to keep your brand top of mind.

---

### **Cluster 2 (Green): "Reward"**
- **Rationale**:  
  This cluster represents your loyal, average customers. Their purchasing behavior aligns closely with the population mean, indicating consistent and reliable engagement over time.  
- **Action**:  
  - Implement a robust loyalty program to reward their commitment.  
  - Provide exclusive offers, early access to products, or recognition to reinforce their loyalty.  
  - Focus on keeping them satisfied to prevent churn.


## Outliers Clustering

In [ ]:
# Find overlapping indices between total and frequency since we already know that they may overlap
overlap_indices = total_outliers.index.intersection(frequency_outliers.index)

# Separate outliers
total_only_outliers = total_outliers.drop(overlap_indices)
frequency_only_outliers = frequency_outliers.drop(overlap_indices)
total_and_frequency_outliers = total_outliers.loc[overlap_indices]

# Manually assign cluster labels
total_only_outliers['Cluster'] = -1 # Negative label to indicate that this has been done manually
frequency_only_outliers['Cluster'] = -2
total_and_frequency_outliers['Cluster'] = -3

outlier_clusters_df = pd.concat([total_only_outliers, frequency_only_outliers, total_and_frequency_outliers])
outlier_clusters_df

In [ ]:
cluster_colors = {-1: '#9467bd',
                  -2: '#8c564b',
                  -3: '#e377c2'}

plt.figure(figsize = (12, 18))

plt.subplot(3, 1, 1)
sns.violinplot(x = outlier_clusters_df['Cluster'], y = outlier_clusters_df['Total'], palette = cluster_colors, hue = outlier_clusters_df["Cluster"])
sns.violinplot(y = outlier_clusters_df['Total'], color = 'gray', linewidth = 1.0) # Violin plot of all unclustered data
plt.title('Total by Cluster')
plt.ylabel('Total')

plt.subplot(3, 1, 2)
sns.violinplot(x = outlier_clusters_df['Cluster'], y = outlier_clusters_df['Frequency'], palette = cluster_colors, hue = outlier_clusters_df["Cluster"])
sns.violinplot(y = outlier_clusters_df['Frequency'], color =  'gray', linewidth = 1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')


plt.subplot(3, 1, 3)
sns.violinplot(x = outlier_clusters_df['Cluster'], y = outlier_clusters_df['Recency'], palette = cluster_colors, hue = outlier_clusters_df["Cluster"])
sns.violinplot(y = outlier_clusters_df['Recency'], color = 'gray', linewidth = 1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')

plt.tight_layout()
plt.show()

## Cluster Identification and Actions

### **Cluster -1 (Monetary Outliers): "Engage"**
- **Characteristics**: High spenders but not necessarily frequent buyers. Their purchases are large but infrequent.  
- **Potential Strategy**: Focus on maintaining their loyalty with personalized offers or luxury services that cater to their high spending capacity.  

---

### **Cluster -2 (Frequency Outliers): "Upsell"**
- **Characteristics**: Frequent buyers who spend less per purchase. These customers are consistently engaged but might benefit from upselling opportunities.  
- **Potential Strategy**: Implement loyalty programs or bundle deals to encourage higher spending per visit, given their frequent engagement.  

---

### **Cluster -3 (Monetary & Frequency Outliers): "Big Reward"**
- **Characteristics**: The most valuable outliers, with extreme spending and frequent purchases. They are likely your top-tier customers who require special attention.  
- **Potential Strategy**: Develop VIP programs or exclusive offers to maintain their loyalty and encourage continued engagement.  

In [ ]:
cluster_labels = {
    0: "New Shoppers",
    1: "Re-engage",
    2: "Reward",
    -1: "Engage",
    -2: "Upsell",
    -3: "Big Reward"
}

full_clustering_df = pd.concat([non_outliers, outlier_clusters_df])
full_clustering_df["ClusterLabel"] = full_clustering_df["Cluster"].map(cluster_labels)

cluster_counts = full_clustering_df['ClusterLabel'].value_counts()
full_clustering_df["Total per 100 pounds"] = full_clustering_df["Total"] / 100.00
feature_means = full_clustering_df.groupby('ClusterLabel')[['Recency', 'Frequency', 'Total per 100 pounds']].mean()

fig, ax1 = plt.subplots(figsize=(15, 10))

sns.barplot(x = cluster_counts.index, y = cluster_counts.values, ax = ax1, palette = 'viridis', hue = cluster_counts.index)
ax1.set_ylabel('Number of Customers', color = 'b')
ax1.set_title('Cluster Distribution with Average Feature Values')

ax2 = ax1.twinx()

sns.lineplot(data = feature_means, ax = ax2, palette = 'Set2', marker = 'o')
ax2.set_ylabel('Average Value', color = 'g')
plt.show()

## Considerations:
1. The majority of clients fall into the **"Reward"** category.  
2. As expected, the **"New Shoppers"** are those who have made recent purchases.  
3. The **"Delight"** clients are incredibly valuable, as they are the most frequent and highest-spending customers. Although their numbers are not significant, this can be advantageous, as a smaller group allows for more personalized and careful handling.  
4. The number of **"Re-engage"** (old) clients is similar to that of **"New Shoppers"**, indicating that action is needed to retain or recapture their interest.  
5. Since the number of **"Upsell"** clients is very small, investing in this group may not yield significant returns.  